In [5]:
import pandas as pd
from dateutil import parser
import re


In [52]:
local = pd.read_csv("c:/users/hp/Downloads/jobs.csv")
local["Position"] = local["Position"].astype(str)
local["Location"] = local["Location"].astype(str)
local["Registration Date"]=local["Registration Date"].str.lower()
local["Scrape date"]= pd.to_datetime(local["Scrape date"], errors="coerce")

def normalizae_position(title):
    if not isinstance(title, str):
        return None
    t = title.lower()

    #Remove any '()' and the content inside 
    t= re.sub(r"\(.*?\)", "", t)

    #Remove puntuations and unccessary special charachters
    t = re.sub(r"\s*[^a-z0-9/&]+\s*", " ", t).strip()

    #Remove unneccary attributes
    t=re.sub(r"\bjr\b\s*", "", t)
    t=re.sub(r"\bjunior\b\s*", "", t)
    t=re.sub(r"\badjunct\b\s*", "", t)
    t=re.sub(r"\bassistant\b\s*", "", t)

   # Remove locations from the position rows
    locations = ["gorgora", "hawassa", "wolkite",
             "arba minch", "bahir dar", "adama",
             "gonder", "mekelle"]

    # Create regex pattern: "gorgora|hawassa|wolkite|arba minch|..."
    pattern = r"-\s*(" + "|".join(locations) + r")\s*$"

    t = re.sub(pattern, "", t)

    return t

local["Position"]=local["Position"].apply(normalizae_position)
'''----------------------------------------------------------------------------------------'''
def normalized_location(text):
    if not isinstance(text, str):
        return None

    text_lower= text.lower()
    matches=[]
    
    keywords = {
    # keyword: return_value
        "e-mail": "online",
        "email": "online",
        "ethiopian": "addis ababa",
        "nekemte": "nekemte", 
        "mekelle": "mekelle",
        "wollo": "wollo",
        "addis": "addis ababa",
        "different": "multiple locations"
    }
    for keyword, display in keywords.items():
        if keyword in text_lower:
            matches.append(display)
    return ", ".join(sorted(set(matches))) if matches else ""
local["Location"] = local["Location"].apply(normalized_location)
'''----------------------------------------------------------------------------------------'''
def split_range(text):
    text = str(text)
    parts = re.split(r"\s+to\s+", text, maxsplit=1, flags=re.IGNORECASE)
    
    if len(parts) == 2:
        return parts[0], parts[1]   # start, end
    else:
        return parts[0], None       # no “to” → only start
        

local["start_raw"], local["end_raw"] = zip(*local["Registration Date"].apply(split_range))
'''----------------------------------------------------------------------------------------'''
def try_parse(t):
    try:
        return parser.parse(str(t), fuzzy=True)
    except:
        return None

local["registration_start_date"] = local["start_raw"].apply(try_parse)
local["registration_end_date"]   = local["end_raw"].apply(try_parse)
'''----------------------------------------------------------------------------------------'''

def duration(row):
    start = row["registration_start_date"]
    end   = row["registration_end_date"]

    # If either date is missing → return None
    if pd.isna(start) or pd.isna(end):
        return None

    # Otherwise compute difference in days
    return (end - start).days

local["duration_days"] = local.apply(duration, axis=1)

def compute_lead_time(row):
    start = row["registration_start_date"]
    scrape = row["Scrape date"]

    # Conditions where we cannot compute a meaningful number
    if pd.isna(start) or pd.isna(scrape):
        return None

    # Registration already started before the scrape
    if start < scrape:
        return None

    # Valid case: start_date >= scrape_date
    return (start - scrape).days


local["lead_time_days"] = local.apply(compute_lead_time, axis=1)
'''----------------------------------------------------------------------------------------'''
local.drop(columns=["start_raw", "end_raw"], errors="ignore", inplace=True)

local.to_csv("normalizaed_jobs.csv", index=False)